<a href="https://colab.research.google.com/github/BethGomez44/proyectoDataScience/blob/main/ProyectoDataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Módulo 2. Análisis de Datos con Python
#### Tema: Análisis de Sentimientos en Reseñas de Películas
#### Objetivo:
